In [72]:
import os
import wget
import numpy as np
from tqdm import tqdm
from scipy.special import expit

if not os.path.exists('hw4_train.dat'):
    train_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw4_train.dat"
    test_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw4_test.dat"
    wget.download(train_url)
    wget.download(test_url)
train_data = np.loadtxt('hw4_train.dat')
test_data = np.loadtxt('hw4_test.dat')
x = train_data[:, :-1]
y = train_data[:, -1].reshape(-1, 1)
x = np.insert(x, 0, values=1, axis=1)
x_test = test_data[:, :-1]
y_test = test_data[:, -1].reshape(-1, 1)
x_test = np.insert(x_test, 0, values=1, axis=1)

In [74]:
def experiment(index=10):
    lam = 10 ** index
    x_ = np.sqrt(lam) * np.identity(x.shape[1])
    y_ = np.zeros((x_.shape[0], 1))
    w = np.linalg.inv(x.T.dot(x) + x_.T.dot(x_)).dot(x.T.dot(y) + x_.T.dot(y_))
    y_predict = np.sign(x.dot(w))
    e_in = np.sum(y_predict != y) / y.size
    y_test_predict = np.sign(x_test.dot(w))
    e_out = np.sum(y_test_predict != y_test) / y_test.size
    return e_in, e_out


# 18
print(experiment(0))
#20
print(experiment(-8))


(0.035, 0.02)
(0.015, 0.02)


In [ ]:
for index in range(2, -11, -1):
    lam = 10 ** index
    print("index", index, end=" ")
    print("e_in e_out", experiment(lam))


In [69]:
x = train_data[:120, :-1]
y = train_data[:120, -1].reshape(-1, 1)
x = np.insert(x, 0, values=1, axis=1)
x_val = train_data[120:, :-1]
y_val = train_data[120:, -1].reshape(-1, 1)
x_val = np.insert(x_val, 0, values=1, axis=1)


def experiment16(lam=10):
    x_ = np.sqrt(lam) * np.identity(x.shape[1])
    y_ = np.zeros((x_.shape[0], 1))
    w = np.linalg.inv(x.T.dot(x) + x_.T.dot(x_)).dot(x.T.dot(y) + x_.T.dot(y_))
    y_predict = np.sign(x.dot(w))
    e_in = np.sum(y_predict != y) / y.size
    y_val_predict = np.sign(x_val.dot(w))
    e_val = np.sum(y_val_predict != y_val) / y_val.size
    y_test_predict = np.sign(x_test.dot(w))
    e_out = np.sum(y_test_predict != y_test) / y_test.size
    return e_in, e_val, e_out


for index in range(2, -11, -1):
    lam = 10 ** index
    print("index", index, end=" ")
    print("e_in e_val e_out", experiment16(lam))

index 2 e_in e_val e_out (0.3416666666666667, 0.4125, 0.414)
index 1 e_in e_val e_out (0.075, 0.125, 0.08)
index 0 e_in e_val e_out (0.03333333333333333, 0.0375, 0.028)
index -1 e_in e_val e_out (0.03333333333333333, 0.0375, 0.022)
index -2 e_in e_val e_out (0.03333333333333333, 0.0375, 0.021)
index -3 e_in e_val e_out (0.03333333333333333, 0.0375, 0.021)
index -4 e_in e_val e_out (0.03333333333333333, 0.0375, 0.021)
index -5 e_in e_val e_out (0.03333333333333333, 0.0375, 0.021)
index -6 e_in e_val e_out (0.03333333333333333, 0.0375, 0.021)
index -7 e_in e_val e_out (0.03333333333333333, 0.0375, 0.021)
index -8 e_in e_val e_out (0.0, 0.05, 0.025)
index -9 e_in e_val e_out (0.0, 0.1, 0.038)
index -10 e_in e_val e_out (0.008333333333333333, 0.125, 0.04)


In [71]:
def experiment(lam, x_train, y_train, x_val, y_val):
    x_ = np.sqrt(lam) * np.identity(x_train.shape[1])
    y_ = np.zeros((x_.shape[0], 1))
    w = np.linalg.inv(x_train.T.dot(x_train) + x_.T.dot(x_)).dot(x_train.T.dot(y_train) + x_.T.dot(y_))
    y_predict = np.sign(x_train.dot(w))
    e_in = np.sum(y_predict != y_train) / y_train.size
    y_val_predict = np.sign(x_val.dot(w))
    e_val = np.sum(y_val_predict != y_val) / y_val.size
    return e_in, e_val


def cv_experiment(lam):
    x = train_data[:, :-1]
    y = train_data[:, -1].reshape(-1, 1)
    x = np.insert(x, 0, values=1, axis=1)
    part = 40
    n = int(y.size / part)
    e_sum = 0
    for i in range(n):
        left = part * i
        right = part * (i + 1)
        val_index = np.s_[left:right]
        x_train = np.delete(x, val_index, axis=0)
        y_train = np.delete(y, val_index, axis=0)
        x_val = x[val_index]
        y_val = y[val_index]
        e_in, e_val = experiment(lam, x_train, y_train, x_val, y_val)
        e_sum += e_val
    e_cv = e_sum / n
    return e_cv


for index in range(2, -11, -1):
    lam = 10 ** index
    print("index", index, end=" ")
    print("e_cv", cv_experiment(lam))




index 2 e_cv 0.29000000000000004
index 1 e_cv 0.06
index 0 e_cv 0.034999999999999996
index -1 e_cv 0.034999999999999996
index -2 e_cv 0.034999999999999996
index -3 e_cv 0.034999999999999996
index -4 e_cv 0.034999999999999996
index -5 e_cv 0.034999999999999996
index -6 e_cv 0.034999999999999996
index -7 e_cv 0.034999999999999996
index -8 e_cv 0.03
index -9 e_cv 0.05
index -10 e_cv 0.05


In [ ]:
a = np.ones((3, 10))
b = np.delete(a, np.s_[1:8], axis=1)
print(b.shape)